In [141]:
%matplotlib inline
import pandas as pd
import pandas_profiling as pp
import numpy as np # linear algebra
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MultiLabelBinarizer 
import seaborn as sns
import tkinter
import matplotlib
import matplotlib.colors
import matplotlib.pyplot as plt

In [142]:
data = pd.read_csv('final_dataset.csv')
colors_amazon = ['#0F79AF', '212529', '343A40']
color_netflix = ['#E50914']
colors = ['#fca311', ]

# What kind of content is present on Streaming Platforms? 

### Tv Series/Movie distribution sunburst
We plot in a sunbarst the distribution of content typology in our dataset (Movie, TV Show) and the consequent subdivision between Amazon and Netflix.

In [143]:
x= data.groupby(['type'])[['amazon', 'netflix']].sum()
y=len(data)
r=((x*100/y)).round(2)
ratio = pd.DataFrame(r)
ratio
df = ratio.stack().reset_index()
df.columns = ['type','distr','val']
df['distr'] = df['distr'].apply(lambda x: x.capitalize())

,type,distr,val
0,Movie,Amazon,35.80
1,Movie,Netflix,43.78
2,TV Show,Amazon,5.85
3,TV Show,Netflix,16.37


In [144]:
fig = px.sunburst(df, path=['distr', 'type'], values='val',color='distr', hover_name="type", color_discrete_map={'Netflix':'#E50914', 'Amazon':'#0F79AF'})
fig.update_traces(marker=dict(line=dict(color='black', width=2)), textfont=dict(family="Helvetica",size=16,color="black"),hoverlabel=dict(font_size=14,font_family="Helvetica",font_color="black"),)
fig.update_traces(hovertemplate=('<br>%{customdata[0]} presence is: %{value}%<br>'))
fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)', hoverlabel_bordercolor='black' )
fig.write_html('visualization/one.html')
fig.show()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



**conclusion:** Netflix is more represented in our dataset than Movies. Surprisingly there are more films than TV series in both platforms

### Country analysis
We plot in a bar chart the 10 countries more represented in our dataset

In [145]:
df = data.copy()
df['country'].replace('no_data', np.nan, inplace  = True)
df.dropna(inplace=True)
country = []
for i, x in df['country'].iteritems():
    l = x.split(', ')
    for c in l:
        country.append(c)
df = pd.DataFrame()
df['country'] = country
df['count'] = 1
df['country'].replace('United States', 'USA', inplace=True)
df['country'].replace('United Kingdom', 'UK',inplace=True)
df['country'].replace('South Korea', 'S. Korea',inplace=True)
x = df.groupby('country')['count'].sum().sort_values(ascending=False)[0:10]
y=len(df)
r=((x*100/y)).round(2)
df = pd.DataFrame(r)

,count
country,
USA,41.00
India,12.98
UK,8.61
Canada,4.90
France,3.91
Germany,2.49
Spain,1.95
Australia,1.79
S. Korea,1.66


In [146]:
fig = go.Figure(data=[go.Bar(x=df.index, y=df['count'].to_list())])
fig.update_traces(marker_color='#fca311', marker_line_color='black',
                  marker_line_width=2)
fig.update_traces(marker=dict(line=dict(color='black', width=2)), textfont=dict(family="Helvetica",size=16,color="black"),hoverlabel=dict(font_size=14,font_family="Helvetica"))
fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)', hoverlabel_bordercolor='black' )
fig.write_html('visualization/two.html')
fig.show()

**Conclusion:** here we can notice that half of the content come from USA, the second most represented country is Idia followed by UK. The huge presence of American content could be related to the fact that both company are from USA.

### Languages analysis

We plot in a bar chart the 10 languages most represented on our streaming services (amazon, netflix).

In [147]:
df = data.copy()
df['languages'].replace('no_data', np.nan, inplace  = True)
df.dropna(inplace=True)
languages = []
for i, x in df['languages'].iteritems():
    l = x.split(', ')
    for lang in l:
        languages.append(lang)
df = pd.DataFrame()
df['languages'] = languages
df['count'] = 1
x = df.groupby('languages')['count'].sum().sort_values(ascending=False)[0:10]
y=len(df)
r=((x*100/y)).round(2)
df = pd.DataFrame(r)

,count
languages,
English,51.48
Hindi,7.81
Spanish,5.61
French,5.08
German,2.66
Tamil,2.14
Mandarin,1.68
Arabic,1.67
Italian,1.61


In [148]:
fig = go.Figure(data=[go.Bar(x=df.index, y=df['count'].to_list())])
fig.update_traces(marker_color='#fca311', marker_line_color='black',
                  marker_line_width=2)
fig.update_traces(marker=dict(line=dict(color='black', width=2)), textfont=dict(family="Helvetica",size=16,color="black"),hoverlabel=dict(font_size=14,font_family="Helvetica"))
fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)', hoverlabel_bordercolor='black' )
fig.write_html('visualization/three.html')
fig.show()

**Conclusion:** as we aspected from countries results, English is the most spoken language on streaming service. Most of the asian films tend to use English languages to be international and placed in this kind of world wide services, even if Hindi is having a good representation in respect to the others. European languages as well have a good rapresentation.

## Rating analysis

We plot in a bar chart ratings present on our dataset divided for type (Movie and TV Show).

In [149]:
df = data.copy()
df['rating'].replace('no_data', np.nan, inplace  = True)
df.dropna(inplace=True)
df['count'] = 1
order = pd.DataFrame(df.groupby('rating')['count'].sum().sort_values(ascending=False).reset_index())
rating_order = list(order['rating'])
mf = df.groupby('type')['rating'].value_counts().unstack().sort_index().fillna(0).astype(int)[rating_order]
movie = mf.loc['Movie']
tv = - mf.loc['TV Show']

rating,Adults 18+,Young Adults 16+,Teens 13+,Older Kids 7+,Kids All,Unrated
type,,,,,,
Movie,2895,1230,1344,645,266,102
TV Show,759,494,64,309,184,18


In [150]:
fig = go.Figure()
fig.add_trace(go.Bar(x=mf.columns, y=movie,
                base=0,
                marker_color='#fca311',
                name='Movie'))
fig.add_trace(go.Bar(x=mf.columns,  y= mf.loc['TV Show'],
                base=[-759, -494, -64, -309, -184, -18],
                marker_color='#495057',
                name='TV Shows'
                ))
fig.update_traces(marker=dict(line=dict(color='black', width=2)), textfont=dict(family="Helvetica",size=16,color="black"),hoverlabel=dict(font_size=14,font_family="Helvetica"))
fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)', hoverlabel_bordercolor='black' )
fig.write_html('visualization/four.html')
fig.show()

**Conclusion:** Here we can notice that most of the contents are made for Adults. Another interesting insights is the fact that movie present more teen than young adoults rating and TV Shows tent to be rated more as Young Adults than Teen.

### Director gender analysis

We plot a pie chart to understand gender distribution in our dataset. Then, in order to better get the evolution during time of gender inclusion we plot a filled area chart.

In [151]:
df = data.copy()
df['director_gender'].replace('no_data', np.nan, inplace  = True)
df = df.dropna(subset=['director_gender'])
gender = []
release_year = []
for i, x in df['director_gender'].iteritems():
    x = x.split(', ')
    for gen in x:
        gender.append(gen)
        year = df._get_value(i, 'release_year')
        release_year.append(year)

df = pd.DataFrame()
df['release_year'] = release_year
df['gender'] = gender
df['count'] = 1
df1 = pd.DataFrame(df.groupby('gender')['count'].sum().sort_values(ascending=False).reset_index())
gender_order = list(df1['gender'])

,gender,count
0,male,5975
1,female,886
2,unknown,256
3,transgender female,4
4,non-binary,2


In [152]:
fig = px.pie(df1, values='count', names='gender')
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig.update_traces(textposition='inside', textinfo='percent+label', marker=dict(colors=colors, line=dict(color='black', width=2)), textfont=dict(family="Helvetica",size=16,color="black"),hoverlabel=dict(font_size=14,font_family="Helvetica"))

fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)', hoverlabel_bordercolor='black' )
fig.write_html('visualization/five.html')
fig.show()

In [153]:
df2 = df[['release_year', 'gender']].groupby('gender')['release_year'].value_counts().unstack().loc[gender_order]
df2 = df2.stack().reset_index()
df2.columns = ['gender','year','val']
df2 = df2[(df2.year >= 2008)&(df2.year < 2022)]
df2 = df2[::-1]
df2

,gender,year,val
188,non-binary,2019.0,1.0
187,non-binary,2013.0,1.0
186,transgender female,2019.0,2.0
185,transgender female,2012.0,1.0
184,transgender female,2009.0,1.0
183,unknown,2021.0,28.0
182,unknown,2020.0,39.0
181,unknown,2019.0,31.0
180,unknown,2018.0,30.0
179,unknown,2017.0,39.0


In [154]:
fig = px.area(df2, x="year", y="val", color="gender", line_shape='spline', color_discrete_map= {'male': '#fca311', 'female':'#E50914', 'unknown': '#212529'}, labels={'gender':'Director gender', 'val':'Director amount', 'year': 'Movie release year'})
fig.update_traces( marker=dict(line=dict(width=2)),textfont=dict(family="Helvetica",size=16,color="black"),hoverlabel=dict(font_size=14,font_family="Helvetica"))
fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)', hoverlabel_bordercolor='black' )
fig.write_html('visualization/six.html')
fig.show()

**Conclusion:** From the two plot we notice that male directors are still the majority. Interestingly, after 2014 (the year of the real boom of streaming services), female directors have seen an increase. But, in the same year the male directors also had an exponential growth. Are we really achieving a gender balance in the world of cinema, or are we simply producing more content in less time?

# How Streaming Services treats their content? 
### Year added
We plot two line chart one for Netflix and one for Amazon to investigate movement by year in content type and quantity addition. 

In [155]:
dfn = data.copy()
dfn['date_added_netflix'].replace('no_data', np.nan, inplace  = True)
dfn = dfn.dropna()
dfn['date_added_netflix'] = pd.to_datetime(dfn['date_added_netflix'])
dfn['year_added'] = dfn['date_added_netflix'].dt.year
dfn['count'] = 1

dfa = data.copy()
dfa['date_added_amazon'].replace('no_data', np.nan, inplace  = True)
dfa = dfa.dropna()
dfa['date_added_amazon'] = pd.to_datetime(dfa['date_added_amazon'])
dfa['year_added'] = dfa['date_added_amazon'].dt.year
dfa['count'] = 1


In [156]:
c = ('rgba(184, 247, 212, 1)', 'rgba(100,0,255,1)')

def year_added(df, file, prov, range):
    fig = go.Figure(layout_yaxis_range=range)
    for i, r in enumerate(df['type'].value_counts().index[::-1]):
        mtv_rel = pd.DataFrame(df[df['type']==r]['year_added'].value_counts().sort_index())
        fig.add_trace(go.Scatter(
            x=mtv_rel.index, y=mtv_rel['year_added'],
            hoverinfo='x+y',
            mode='lines',
            line=dict(width=2, color=c[i]),
            # line_shape='spline', 
            name = r,
            stackgroup='one' # define stack group
        ))

    fig.update_traces( marker=dict(line=dict(width=2)),textfont=dict(family="Helvetica",size=16,color="black"),hoverlabel=dict(font_size=14,font_family="Helvetica"))
    fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)', hoverlabel_bordercolor='black',xaxis_title="Contents "+ prov +" addition year",
    yaxis_title="Contents amount",
    legend_title="Contents type",
    font=dict(
        family="Helvetica, San serif",
        size=12,
        color="black"
    ) )
    fig.write_html('visualization/'+ file)
    fig.show()
year_added(dfn, 'seven.html', 'Netflix', [0,1300])
year_added(dfa, 'eight.html', 'Amazon', [0,300])

**Conclusions:** Netflix and Amazon act in different ways in matter of content addition. If on one heand Netflix appear to have an increase of contents addition after 2014, on the other heand, Amazon seems to be more constant but it show less offer. Both the streaming services have a down trand after 2020, probably due to Covid-19 pandemic

### Month added 

We plot two line chart one for Netflix and one for Amazon to investigate movement by month in content type and quantity addition. 

In [157]:
month_order = ['January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December']

dfn = data.copy()
dfn['date_added_netflix'].replace('no_data', np.nan, inplace  = True)
dfn = dfn.dropna()
dfn['date_added_netflix'] = pd.to_datetime(dfn['date_added_netflix'])
dfn['month_added_netflix']=dfn['date_added_netflix'].dt.month
dfn['month_added_netflix']=dfn['date_added_netflix'].dt.month_name()
dfn['month_name_added'] = pd.Categorical(dfn['month_added_netflix'], categories=month_order, ordered=True)

dfa = data.copy()
dfa['date_added_amazon'].replace('no_data', np.nan, inplace  = True)
dfa = dfa.dropna()
dfa['date_added_amazon'] = pd.to_datetime(dfa['date_added_amazon'])
dfa['month_added_amazon']=dfa['date_added_amazon'].dt.month
dfa['month_added_amazon']=dfa['date_added_amazon'].dt.month_name()
dfa['month_name_added'] = pd.Categorical(dfa['month_added_amazon'], categories=month_order, ordered=True)

In [158]:
c = ('rgba(184, 247, 212, 1)', 'rgba(100,0,255,1)')

def month_added(df, file, prov, range):
    data_sub = df.groupby('type')['month_name_added'].value_counts().unstack().fillna(0).loc[['TV Show','Movie']].cumsum(axis=0).T
    fig = go.Figure(layout_yaxis_range=range)
    for i, r in enumerate(df['type'].value_counts().index[::-1]):
        mtv_rel = pd.DataFrame(data_sub[r])
        fig.add_trace(go.Scatter(
            x=mtv_rel.index, y=mtv_rel[r],
            hoverinfo='x+y',
            mode='lines',
            line=dict(width=2, color=c[i]),
            # line_shape='spline', 
            name = r,
            stackgroup='one' # define stack group
        ))

    fig.update_traces( marker=dict(line=dict(width=2)),textfont=dict(family="Helvetica",size=16,color="black"),hoverlabel=dict(font_size=14,font_family="Helvetica"))
    fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)', hoverlabel_bordercolor='black',xaxis_title="Contents "+ prov +" addition month",
    yaxis_title="Contents amount",
    legend_title="Contents type",
    font=dict(
        family="Helvetica, San serif",
        size=12,
        color="black"
    ) ) 
    fig.write_html('visualization/'+ file)
    fig.show()
month_added(dfn, 'nine.html', 'Netflix', [0,700])
month_added(dfa, 'ten.html', 'Amazon', [0, 200])

**Conclusions:** Interensting is to notice that Amazon and Netflix seems to be complementary in month addition. If Netflix is publishing mosly at the beginning and at the end of the year, Amazon concentrate more in the middle - in particular on the moth of June. 

### Content genre addition over month

We plot two histogram to understand if there is a differenciation in terms of content's genres month addition between the two streaming serivicies. 

In [161]:
month_order = ['January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December']

dfn = data.copy()
dfn['date_added_netflix'].replace('no_data', np.nan, inplace  = True)
dfn = dfn.dropna()
dfn['date_added_netflix'] = pd.to_datetime(dfn['date_added_netflix'])
dfn['month_added_netflix']=dfn['date_added_netflix'].dt.month
dfn['month_added_netflix']=dfn['date_added_netflix'].dt.month_name()
dfn['month_name_added'] = pd.Categorical(dfn['month_added_netflix'], categories=month_order, ordered=True)
dfn['genre'] = dfn['listed_in'].apply(lambda x :  x.replace(' ,',',').replace(', ',',').split(',')) 

dfa = data.copy()
dfa['date_added_amazon'].replace('no_data', np.nan, inplace  = True)
dfa = dfa.dropna()
dfa['date_added_amazon'] = pd.to_datetime(dfa['date_added_amazon'])
dfa['month_added_amazon']=dfa['date_added_amazon'].dt.month
dfa['month_added_amazon']=dfa['date_added_amazon'].dt.month_name()
dfa['month_name_added'] = pd.Categorical(dfa['month_added_amazon'], categories=month_order, ordered=True)
dfa['genre'] = dfa['listed_in'].apply(lambda x :  x.replace(' ,',',').replace(', ',',').split(',')) 

In [166]:
def genre_month(df, file, prov):
    d = pd.DataFrame()
    j = 0
    for i, x in df['genre'].iteritems():
        for gen in x:
            d.loc[j, 'genre'] = gen
            mh = df._get_value(i, 'month_name_added')
            d.loc[j, 'month'] = mh
            d.loc[j, 'count'] = 1
            j+=1
    d = d[['month', 'genre']].groupby('genre')['month'].value_counts().unstack()
    d = d.stack().reset_index()
    d.columns = ['genre','month','count']
    fig = px.histogram(d, x="month", y="count", color='genre', category_orders=dict(month=month_order))
    fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)', hoverlabel_bordercolor='black',xaxis_title="Contents divided by genre on "+ prov,
    yaxis_title="Contents amount",
    legend_title="Genre",
    font=dict(
        family="Helvetica, San serif",
        size=12,
        color="black"
    ) ) 
    fig.write_html('visualization/'+ file)
    fig.show()

genre_month(dfn, 'eleven.html', 'Netflix')
genre_month(dfa, 'twelve.html', 'Amazon')

**Conclusion:** both the servicies have a dominance of Drammatic and Commedian contents. Amazon have a huge ammount of international movies, in respect to Amazon, that appear to add constantly by month. etc... (se hai tempo scrivi) 

### Genere correlation

In order to understand how genres combine together for type and streaming services we built a heatmap.

In [192]:
df = data.copy()
df['listed_in'].replace('no_data', np.nan, inplace  = True)
df = df.dropna(subset=['listed_in'])
df_tv = df[df["type"] == "TV Show"]
df_movies = df[df["type"] == "Movie"]

df_amazon = df.query("amazon == 1.0")
df_tv_a = df_amazon[df_amazon["type"] == "TV Show"]
df_movies_a = df_amazon[df_amazon["type"] == "Movie"]

df_netflix = df.query("netflix == 1.0")
df_tv_n = df_netflix[df_netflix["type"] == "TV Show"]
df_movies_n = df_netflix[df_netflix["type"] == "Movie"]
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#221f1f', '#b20710','#f5f5f1'])
color_list = [matplotlib.colors.rgb2hex(cmap(i)) for i in range(cmap.N)]

In [193]:
def genre_heatmap(df, file, prov):
    df['genre'] = df['listed_in'].apply(lambda x :  x.replace(' ,',',').replace(', ',',').split(',')) 
    test = df['genre']
    mlb = MultiLabelBinarizer()
    res = pd.DataFrame(mlb.fit_transform(test), columns=mlb.classes_, index=test.index)
    corr = res.corr()
    mask = np.zeros_like(corr, dtype = bool) #probably could do this on df… to check
    mask[np.triu_indices_from(mask)] = True
    corr1 = corr.mask(mask)
    X = corr1.columns.values
    corr = corr1.values.tolist()
    N = len(corr)
    hovertext = [[f'corr({X[i]}, {X[j]})= {corr[i][j]:.2f}' if i>j else '' for j in range(N)] for i in range(N)]
    title = prov + ' genres' 
    heat = go.Heatmap(z=corr,
                  x=X,
                  y=X,
                  xgap=1, ygap=1,
                  colorscale=color_list,
                  colorbar_thickness=20,
                  colorbar_ticklen=3,
                  hovertext =hovertext,
                  hoverinfo='text'
                   )  
    layout = go.Layout(
                title_text=title, title_x=0.5,
                   width=900, height=900,
                   xaxis_showgrid=False,
                   yaxis_showgrid=False,
                   yaxis_autorange='reversed')
    fig=go.Figure(data=[heat], layout=layout)   
    fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)', hoverlabel_bordercolor='black',xaxis_title="Contents divided by genre on "+ prov,
    yaxis_title="Contents amount",
    legend_title="Genre",
    font=dict(
        family="Helvetica, San serif",
        size=12,
        color="black"
    ) ) 
    fig.write_html('visualization/'+ file)     
    fig.show() 

genre_heatmap(df, 'therteen.html', 'Full dataset')
genre_heatmap(df_tv, 'fourteen.html', 'Full tv shows')
genre_heatmap(df_movies, 'fifteen.html', 'Full movies')

genre_heatmap(df_amazon, 'sixteen.html', 'Full Amazon')
genre_heatmap(df_tv_a, 'seventeen.html', 'Amazon tv shows')
genre_heatmap(df_movies_a, 'eighteen.html', 'Amazon movies')

genre_heatmap(df_netflix, 'nineteen.html', 'Full Netflix')
genre_heatmap(df_tv_n, 'twenty.html', 'Netflix tv shows')
genre_heatmap(df_movies_n, 'twentyone.html', 'Netflix movies')

/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/3366203672.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/3366203672.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/3366203672.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/3366203672.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/3366203672.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/3366203672.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/3366203672.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/3366203672.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**Conclusion:** scrivere qualcosa poi 

### Cinema culture representation

To understand how old are movie present in sreaming services, and which country cinema culture is more represented in our dataset we plot --- one for Amazon and one from Netflix.

In [289]:
df = data.copy()
df['country'].replace('no_data', np.nan, inplace  = True)
df = df.dropna()
df['first_country'] = df['country'].apply(lambda x: x.split(",")[0])
df['first_country'].replace('United States', 'USA', inplace=True)
df['first_country'].replace('United Kingdom', 'UK',inplace=True)
df['first_country'].replace('South Korea', 'S. Korea',inplace=True)
df['count'] = 1

dfn = df.copy()
dfn['date_added_netflix'].replace('no_data', np.nan, inplace  = True)
dfn = dfn.dropna()
dfn['date_added_netflix'] = pd.to_datetime(dfn['date_added_netflix'])
dfn['year_added'] = dfn['date_added_netflix'].dt.year
df_movies_n = dfn[dfn["type"] == "Movie"]
df_series_n = dfn[dfn["type"] == "Movie"]

dfa = df.copy()
dfa['date_added_amazon'].replace('no_data', np.nan, inplace  = True)
dfa = dfa.dropna()
dfa['date_added_amazon'] = pd.to_datetime(dfa['date_added_amazon'])
dfa['year_added'] = dfa['date_added_amazon'].dt.year
df_movies_a = dfa[dfa["type"] == "Movie"]
df_series_a = dfa[dfa["type"] == "Movie"]


In [291]:
def scatter_repr(data):
    df = data.groupby('first_country')[['first_country','count']].sum().sort_values(by='count',ascending=False).reset_index()[:10]
    df = df['first_country']
    df_loli = data.loc[data['first_country'].isin(df)]
    loli = df_loli.groupby('first_country')['release_year','year_added'].mean().round()
    ordered_df = loli.sort_values(by='release_year')
    my_range=[0,len(loli.index)+1]

    min = ordered_df['release_year'].to_list()[0]-3
    
    label_pos = [2022] * len(ordered_df+1)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x= label_pos,
        y= list(range(1,11)),
        text= ordered_df.index.values.tolist(),
        mode="text",
        hoverinfo='skip',
        showlegend=False,
    ))

    fig.add_trace(go.Scatter(
            x=ordered_df['release_year'].to_list(),
            y=list(range(1,11)),
            marker=dict(color="gold", size=12),
            mode="markers",
            name="Release year",
    ))

    fig.add_trace(go.Scatter(
            x=ordered_df['year_added'].to_list(),
            y=list(range(1,11)),
            marker=dict(color="blue", size=12),
            mode="markers",
            name="Added year",
    ))

    fig.update_layout(yaxis_range=my_range, xaxis_range=[min,2024],
        xaxis_showgrid=False,
        yaxis_showgrid=False,
        yaxis_showticklabels=False,
        paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)', hoverlabel_bordercolor='black',
        font=dict(
            family="Helvetica, San serif",
            size=12,
            color="black"
        ) 
    )

    y_pos = 1
    for i, x in ordered_df.iterrows():

        fig.add_shape(type='line',
                        x0=ordered_df['release_year'][i],
                        y0=y_pos,
                        x1=ordered_df['year_added'][i],
                        y1=y_pos,
                        line=dict(color='Red',),
                        xref='x',
                        yref='y',
                        layer='below'
        )
        y_pos += 1

    fig.show()
scatter_repr(df_movies_n)
scatter_repr(df_series_n)
scatter_repr(df_movies_a)
scatter_repr(df_series_a)

/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/1185624427.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/1185624427.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/1185624427.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



/var/folders/3g/t_1rwv_57vz466tvccx4d6gm0000gn/T/ipykernel_3485/1185624427.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

